2. Data Analysis (12 points)

Run at least one statistical test (e.g., t-test for exposure change around key event)

In [ ]:
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt

df = pd.read_csv(r"/content/brexit_stats.csv")

df['date'] = pd.to_datetime(df[['year', 'month', 'day']])
cutoff_date = pd.Timestamp('2016-06-23')

pre_brexit = df[df['date'] < cutoff_date]['BrexitExposure']
post_brexit = df[df['date'] >= cutoff_date]['BrexitExposure']

t_statistic, p_value = stats.ttest_ind(pre_brexit, post_brexit, equal_var=False)

print(f"T-statistic: {t_statistic}")
print(f"P-value: {p_value}")

A two-sample t-test assuming unequal variances was conducted to determine if there was a statistically significant difference in firm-level Brexit exposure before and after the referendum on June 23, 2016. The analysis yielded a t-statistic of -5.83 and a p-value of  1.41×10−8 . Given that the p-value is well below the conventional alpha level of 0.05, we reject the null hypothesis that the mean exposure levels are equal between the two periods.

The negative t-statistic indicates that the mean exposure in the pre-referendum period was significantly lower than in the post-referendum period. This confirms that the Brexit vote acted as a major structural break, leading to a substantial and sustained increase in Brexit-related risk discussions during firm earnings calls. These findings support the hypothesis that the referendum introduced significant uncertainty into the global market, compelling firms to devote more attention to Brexit-related topics in their financial disclosures.

3. Interpretation (10 points) Identify key patterns: Which firms/periods show highest exposure? Why?

In [ ]:
df['quarter'] = df['date'].dt.to_period('Q')

sector_map = {
    'BARC.L': 'Financials',
    'F.N': 'Automotive',
    '7203.T': 'Automotive',
    'TSCO.L': 'Consumer & Retail',
    'ULVR.L': 'Consumer & Retail',
    'KR.N': 'Consumer & Retail',
    'KRFT.O': 'Consumer & Retail',
    'KHC.OQ': 'Consumer & Retail',
    'LLY.N': 'Healthcare',
    'JNJ.N': 'Healthcare',
    'AZN.L': 'Healthcare',
    'DPWGn.DE': 'Industrials & Logistics',
    'KOG.OL': 'Industrials & Logistics',
    'TOM.OL': 'Industrials & Logistics',
    'FMC.N': 'Industrials & Logistics',
    'BP.L': 'Energy'
}

df['quarter'] = df['date'].dt.to_period('Q')

df_sector_trends = df.groupby(['quarter', 'Sector'])['BrexitExposure'].mean().unstack()
df_sector_trends.index = df_sector_trends.index.to_timestamp()

overall_trend = df.groupby('quarter')['BrexitExposure'].mean()
overall_trend.index = overall_trend.index.to_timestamp()

plt.figure(figsize=(12, 7))

for sector in df_sector_trends.columns:
    plt.plot(df_sector_trends.index, df_sector_trends[sector], marker='o', linestyle='-', label=sector, alpha=0.6)

plt.plot(overall_trend.index, overall_trend.values, color='black', linewidth=3, linestyle='-', label='All Sectors (Average)')

plt.axvline(pd.Timestamp('2016-06-23'), color='red', linestyle='--', alpha=0.8, label='Referendum (June 2016)')
plt.axvline(pd.Timestamp('2019-03-29'), color='green', linestyle='--', alpha=0.8, label='Original Deadline (March 2019)')
plt.axvline(pd.Timestamp('2020-12-31'), color='blue', linestyle='--', alpha=0.8, label='Implementation (Dec 2020)')

plt.title('Average Brexit Exposure by Sector Over Time')
plt.xlabel('Date')
plt.ylabel('Brexit Exposure Score')
plt.legend(title='Sector / Event', loc='upper left')
plt.grid(True, alpha=0.3)

plt.show()

Interpretation
Identify key patterns: Which firms/periods show highest exposure? Why? Pattern Identification & Interpretation

A. Temporal Patterns: When was exposure highest? The visualization reveals that Brexit exposure occurred in three distinct waves, each driven by a different type of risk. The first and highest peak appeared in Q3 2016 (The Referendum), representing "outcome uncertainty" as sectors like Financials and Automotive scrambled to assess the shock of the unexpected "Leave" vote. A secondary wave emerged in Q1 2019 (Original Deadline), driven by "cliff-edge anxiety" as firms prepared for a potential "No Deal" crash-out. Finally, a third peak in Q4 2020 (Implementation) reflected "operational risk," where the focus shifted to the practical reality of new customs borders and tariffs as the transition period ended.

Cross-Sectional Patterns: Which firms/sectors were most exposed? The chart highlights a massive divergence between sectors regarding when and how much they reacted:

Financials (e.g., Barclays): Acted as the "Canary in the Coal Mine," showing the most extreme reaction immediately in 2016 due to existential regulatory threats, but adapting relatively quickly.

Retail (e.g., Tesco): Exhibited a "Slow Burn," reacting intensely only when the physical border controls became imminent in 2020.

The "Average" Firm: The black line remains consistently lower than the sector peaks, indicating that while Brexit was a broad market concern, extreme risk exposure was concentrated in specific regulated and trade-dependent sectors.

3. Interpretation: Compare to original paper: Are your findings consistent? Note differences and provide explanations.

A. Areas of Consistency

Our analysis strongly supports the core hypothesis of Hassan et al. (2024) that Brexit was a global uncertainty shock rather than just a domestic UK event. Regarding the global propagation of risk, the original authors find "widespread reverberations in listed firms in 81 countries," arguing that international firms with trade or regulatory links to the UK were highly exposed. This is confirmed by the high exposure of Ford (F.N), a US-domiciled firm, in our analysis. Ford's exposure trajectory mirrored that of UK firms, validating the paper’s conclusion that multinational supply chains transmitted the shock across borders.

Furthermore, our findings regarding transmission channels align perfectly with the literature. The original paper identifies "regulation" and "trade" as primary mechanisms for Brexit risk transmission. Our sector analysis reflects this, as Financials (Barclays) spiked immediately due to regulatory divergence concerns like passporting, while Automotive (Ford) spiked due to trade friction risks. This confirms that the mechanism of exposure depends heavily on the sector's reliance on EU-UK integration. Additionally, the timing of the uncertainty we observed is consistent with the paper's focus on "Brexit Uncertainty" (the second moment) peaking around political events. The massive spike in Q3 2016 in our data matches the paper’s definition of a "second moment" shock—a sudden increase in the variance of possible outcomes immediately following the surprise referendum result.

B. Differences and Nuances

While the general patterns align, our analysis highlights a distinction regarding the end of the Brexit process which the original paper, with its focus on "Uncertainty," emphasizes less. Hassan et al. focus primarily on the fear of the unknown, heavily emphasizing the period following the 2016 vote where firms did not know what Brexit would look like. In contrast, our data shows a distinct secondary peak in Q4 2020 for Retailers (e.g., Tesco). This difference likely represents a shift from "Uncertainty" (Risk) to "Realization" (Impact). By late 2020, firms knew exactly what was happening (leaving the Single Market); the spike in discussion was likely about operational implementation rather than abstract political risk. Our analysis captures this "First Moment" (negative impact) phase more acutely in the retail sector.

Another key difference lies in granularity and sample size. The original paper utilizes a sample of thousands of firms to run regressions controlling for firm size, fixed effects, and other variables, whereas our analysis relies on a simplified sample of approximately 15 firms. While our "Top 5" trends visually match their aggregate findings, our individual firm spikes (e.g., specific volatile quarters for smaller firms) may contain idiosyncratic noise that the original paper averages out.

Conclusion

Our replication successfully validates the "Global Impact" thesis of Hassan et al. (2024). We confirm that exposure was determined not by where a firm is headquartered (US vs. UK), but by its functional reliance on cross-border trade and regulation.

3 Interpretation Business Implications: Disclosure Quality & Risk Communication A. Disclosure Quality: Specificity vs. Boilerplate A key indicator of high "disclosure quality" is that risk reporting should reflect a firm’s unique economic reality rather than being generic "boilerplate" text.

The Pattern: Our analysis shows massive sectoral divergence. Financials spiked in 2016, while Retailers spiked in 2020.

The Implication: This suggests high disclosure quality. If firms were merely copy-pasting generic Brexit warnings to protect themselves legally, we would see uniform patterns across all industries. Instead, the distinct timing proves that management is tailoring their communication to specific, material risks as they arise (e.g., Tesco discussing customs checks only when they became imminent). Investors are receiving idiosyncratic, value-relevant signals rather than noise.

B. Risk Communication: Managing Uncertainty vs. Impact The "Three Waves" of exposure reveal how firms use earnings calls to manage different types of investor anxiety:

Crisis Communication (2016): The immediate spike following the referendum represents "Crisis Communication." In the face of a "Black Swan" event (unexpected outcome), management’s priority was to reduce information asymmetry. By allocating significant airtime to Brexit immediately, firms like Barclays aimed to reassure investors that they were assessing the shock, even before policy details were known.

Operational Guidance (2020): The later peak in Retail represents "Operational Guidance." As the deadline approached, the communication shifted from abstract political risk ("What might happen?") to concrete financial impact ("How much will tariffs cost?"). This transition demonstrates that effective risk communication evolves from explaining volatility to estimating cash flow impacts.

C. The "Global" Narrative The high exposure of US firms (like Ford) highlights a critical challenge in modern investor relations: Cross-Border Narrative Building.

The Implication: Multinational firms cannot treat local political shocks as isolated events. A US-headquartered firm must articulate a coherent risk narrative about a UK political event to global shareholders. The data shows that "Distance" (geographic) matters less than "Function" (supply chain integration). Effective risk communication requires explaining these complex, cross-border transmission channels to investors who may not understand the intricacies of EU trade law.

Conclusion for Management The data suggests that silence is not an option during structural breaks. However, the timing of disclosure is strategic. Firms appear to increase disclosure intensity specifically when the risk transforms from a general market concern into a material operational threat to their specific business model.